In [ ]:
import abc
from EasyGA.parent import Parent
from EasyGA.crossover import Crossover
import pygad
import copy
import numpy as np

Define gene space as numbers from 1 to 9

In [ ]:
sudoku_size = 9
number_of_squares_in_a_line = int(np.sqrt(sudoku_size))

sudoku = np.zeros((sudoku_size, sudoku_size))

Basic inputs

In [ ]:
initial_board = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
])

In [ ]:
error = []

def get_score(array):
    array_converted = array.tolist()
    score = 0
    array_error = 0
    
    for element in array_converted:
        count = array_converted.count(element)
        array_error += count

        if count == 1:
            score += 1
        #else:
        #    score -= 0.025 * count

    error.append(array_error)

    return score

def get_score_of_point(table, position):
    sudoku_chromosome = np.array(table).reshape((sudoku_size,sudoku_size))
    score_position = 0
    
    search_square = np.zeros(sudoku_size * sudoku_size)
    search_square[position] = 1
    search_square = search_square.reshape((number_of_squares_in_a_line, number_of_squares_in_a_line, sudoku_size))

    # print(search_square)
    
    for row_idx, cutted_horizontal_row in enumerate(search_square):
        transposed_squares_searcher = np.split(cutted_horizontal_row.T, number_of_squares_in_a_line)
        # print(transposed_squares_searcher)
        for col_idx, square in enumerate(transposed_squares_searcher):
            if 1 in square:
                # print("rows %s " % row_idx, "cols %s " % col_idx)
                # print(search_square.reshape((sudoku_size, sudoku_size)))
                
                matrix_cutted_in_cols = sudoku_chromosome.reshape((number_of_squares_in_a_line, number_of_squares_in_a_line, sudoku_size))

                # cut number_of_squares_in_a_line in the col indicated
                transposed_square = np.split(matrix_cutted_in_cols[col_idx].T, number_of_squares_in_a_line)

                # print(transposed_square[row_idx].T)
                # print(sudoku_chromosome)
                
                score_position += get_score(transposed_square[row_idx].reshape(sudoku_size))
                score_position += get_score(sudoku_chromosome[row_idx])
                score_position += get_score(sudoku_chromosome.T[col_idx])  # list.T transform rows in cols and cols in rows
    
    return score_position

def crossover_func(parents, offspring_size, ga_instance):
    offspring = []
    parents_reshaped = []
    convert_to_transpose = np.random.random()

    for parent_idx in range(len(parents)):
        parents_reshaped.append(parents[parent_idx].reshape((sudoku_size, sudoku_size)))
        if convert_to_transpose < 0.5:
            parents_reshaped[parent_idx] = parents_reshaped[parent_idx].T 
            
    for children_idx in range(offspring_size[0]):
        sudoku_zeros = np.zeros((sudoku_size, sudoku_size))
        
        for row in range(sudoku_size):
            who_parent_copy = np.random.randint(0, len(parents))
            
            sudoku_zeros[row] = parents_reshaped[who_parent_copy][row]
        
        if convert_to_transpose < 0.5:
            sudoku_zeros = sudoku_zeros.T
            
        offspring.append(sudoku_zeros.reshape(sudoku_size * sudoku_size))
   
    return np.array(offspring)

def fitness_function_impl(solution, solution_index):
    fitness_value = 0
    
    sudoku_chromosome = np.array(solution).reshape((sudoku_size,sudoku_size))

    # print(sudoku_chromosome)
    # add fitness value if it is different between the last number in the cols and rows
    for row_and_col in range(sudoku_size):
        fitness_value += get_score(sudoku_chromosome[row_and_col])
        fitness_value += get_score(sudoku_chromosome.T[row_and_col])  # list.T transform rows in cols and cols in rows

    matrix_cutted_in_cols = sudoku_chromosome.reshape((number_of_squares_in_a_line, number_of_squares_in_a_line, sudoku_size))
    
    for cutted_row in matrix_cutted_in_cols:
        transposed_squares = np.split(cutted_row.T, number_of_squares_in_a_line)

        for transposed_square in transposed_squares:

            square_list = transposed_square.reshape(sudoku_size)
            fitness_value += get_score(square_list) # list.T transform rows in cols and cols in rows

            #square = transposed_square.T
            #print(square)

    return fitness_value

def on_generation(ga_instance):    
    if ga_instance.generations_completed == 3:
        print("Generation", ga_instance.generations_completed)
        print(ga_instance.population)
    
    if ga_instance.generations_completed == 400:
        ga_instance.mutation_probability = [2/(sudoku_size * sudoku_size), 1/(sudoku_size * sudoku_size)]

        
sol_per_pop=30

offspring = np.zeros((sudoku_size + 1))
moda = np.zeros((sudoku_size + 1))

ga_sudoku_instance = pygad.GA(
    num_generations=900,
    num_parents_mating=2,
    fitness_func=fitness_function_impl,
    
    sol_per_pop=sol_per_pop,
    num_genes= sudoku_size * sudoku_size,
    gene_space=np.arange(1, sudoku_size + 1),
    
    keep_elitism=9,
    
    parent_selection_type="sss",
    # parent_selection_type="tournament",
    # K_tournament=10,
    
    keep_parents=2,
    
    # crossover_type= "uniform",
    crossover_type=crossover_func,
    crossover_probability = 0.30,

    mutation_type="adaptive",
    mutation_probability = [7.5/(sudoku_size * sudoku_size), 0.7/(sudoku_size * sudoku_size)], # 218

    stop_criteria=["saturate_360", "reach_"+str(3 * sudoku_size ** 2)],
    #stop_criteria=["reach_"+str(3 * sudoku_size ** 2)],
    gene_type=int,
    
    save_solutions=True,
    save_best_solutions=True,
    # allow_duplicate_genes=False,
    
    random_seed=54,
    parallel_processing=6, # 4
    on_generation=on_generation,
)

print("maximum error value: %s" % (sudoku_size * sudoku_size * 3))

# i made a copy for use later
empty_ga_sudoku_instance = copy.deepcopy(ga_sudoku_instance)

ga_sudoku_instance.run()
print("best fitness value: %s" % (ga_sudoku_instance.best_solutions_fitness[-1]))
print("mean error: %s" % (np.mean(error) - sudoku_size))


print(np.array(ga_sudoku_instance.best_solutions[-1]).reshape((sudoku_size, sudoku_size)))
print()